<a href="https://colab.research.google.com/github/fnuArsh/CV/blob/main/Julia_Colab_Notebook_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2025-02-27 12:50:53 URL:https://julialang-s3.julialang.org/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package CUDA...


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [5]:
versioninfo()

NameError: name 'versioninfo' is not defined

In [2]:
using BenchmarkTools

M = rand(2^11, 2^11)

@btime $M * $M;

  404.042 ms (2 allocations: 32.00 MiB)


In [3]:
try
    using CUDA
catch
    println("No GPU found.")
else
    run(`nvidia-smi`)
    # Create a new random matrix directly on the GPU:
    M_on_gpu = CUDA.CURAND.rand(2^11, 2^11)
    @btime $M_on_gpu * $M_on_gpu; nothing
end

No GPU found.


In [3]:
using ColorSchemes
using ColorTypes
using FileIO
using Images
using Colors
using ImageContrastAdjustment
using Statistics
using StatsBase
using Distributions

function process_c!(counts, c, max_iter, xmin, xmax, ymin, ymax, x_scale, y_scale, width, height)
    z = 0.0 + 0.0im
    escaped = false
    iter_steps = 0

    # First determine if the point escapes and how many steps it takes
    for i in 1:max_iter
        z = z^2 + c
        iter_steps += 1
        if abs2(z) > 4.0
            escaped = true
            break
        end
    end

    # Only process points that escape
    escaped || return

    # Store the trajectory
    trajectory = Vector{Complex{Float64}}(undef, iter_steps)

    # Reset and compute the trajectory
    z = 0.0 + 0.0im
    for i in 1:iter_steps
        z = z^2 + c
        trajectory[i] = z
    end

    # Plot the trajectory with anti-aliasing for smoother results
    for z in trajectory
        x, y = real(z), imag(z)
        if xmin <= x <= xmax && ymin <= y <= ymax
            # Convert to image coordinates
            px_exact = (x - xmin) * x_scale
            py_exact = (ymax - y) * y_scale

            # Integer parts
            px_int = floor(Int, px_exact)
            py_int = floor(Int, py_exact)

            # Fractional parts for anti-aliasing
            fx = px_exact - px_int
            fy = py_exact - py_int

            # Contribute to the four surrounding pixels weighted by distance
            for dx in 0:1
                for dy in 0:1
                    px = px_int + dx + 1
                    py = py_int + dy + 1

                    # Weight by distance (bilinear interpolation)
                    weight = (dx == 0 ? 1-fx : fx) * (dy == 0 ? 1-fy : fy)

                    if 1 <= px <= width && 1 <= py <= height
                        @inbounds counts[py, px] += weight
                    end
                end
            end
        end
    end
end

function buddhabrot()
    # Higher resolution for more detail
    width, height = 2000, 2000

    # Adjust the view area to focus on interesting parts
    xmin, xmax = -2.0, 1.0
    ymin, ymax = -1.5, 1.5

    # Increase iterations for finer detail
    max_iter = 500000

    # Smart sampling for more efficiency
    n_samples = 10000000

    x_scale = (width - 1) / (xmax - xmin)
    y_scale = (height - 1) / (ymax - ymin)

    # Use Float64 for counts to support anti-aliasing
    counts = zeros(Float64, height, width)

    println("Starting sampling...")

    # More efficient sampling focusing on the boundary of the Mandelbrot set
    for i in 1:n_samples
        # Focus sampling around the boundary of the Mandelbrot set
        if rand() < 0.7  # 70% of samples near the boundary
            # Sample from regions known to have interesting detail
            r = 0.1 + 0.9 * rand()  # radius between 0.1 and 1.0
            theta = 2π * rand()     # angle between 0 and 2π

            # Convert to Cartesian coordinates around the main cardioid
            cx = r * cos(theta) - 0.25
            cy = r * sin(theta)

            # Apply some jitter to avoid regular patterns
            cx += 0.2 * (rand() - 0.5)
            cy += 0.2 * (rand() - 0.5)
        else
            # Regular sampling for the rest
            cx = rand() * (xmax - xmin) + xmin
            cy = rand() * (ymax - ymin) + ymin
        end

        c = complex(cx, cy)
        process_c!(counts, c, max_iter, xmin, xmax, ymin, ymax, x_scale, y_scale, width, height)

        # Progress update
        if i % 1000000 == 0
            println("Processed $i samples...")
        end
    end

    println("Processing complete. Generating image...")

    # Apply advanced histogram equalization for more detail
    # First get a more aggressive log transform
    log_counts = log.(counts .+ 1)

    # Apply histogram equalization to bring out details
    equalized = adjust_histogram(log_counts, Equalization(nbins = 2^16))

    # Apply additional contrast stretching
    enhanced = adjust_contrast(equalized, LinearStretching())

    # Orion Nebula inspired gradient with finer gradations
    orion_nebula_gradient = [
        RGB(0.0, 0.0, 0.05),      # Deep space black with hint of blue
        RGB(0.05, 0.02, 0.1),     # Very dark purple
        RGB(0.1, 0.05, 0.2),      # Dark purple
        RGB(0.15, 0.03, 0.25),    # Intermediate
        RGB(0.2, 0.0, 0.3),       # Deep magenta
        RGB(0.3, 0.0, 0.35),      # Intermediate
        RGB(0.4, 0.0, 0.4),       # Rich purple
        RGB(0.5, 0.05, 0.35),     # Intermediate
        RGB(0.6, 0.1, 0.3),       # Dark reddish purple
        RGB(0.7, 0.15, 0.25),     # Intermediate
        RGB(0.8, 0.2, 0.2),       # Deep red
        RGB(0.9, 0.3, 0.15),      # Intermediate
        RGB(1.0, 0.4, 0.1),       # Bright orange/red
        RGB(1.0, 0.5, 0.2),       # Intermediate
        RGB(1.0, 0.6, 0.3),       # Golden orange
        RGB(0.95, 0.7, 0.45),     # Intermediate
        RGB(0.9, 0.8, 0.6),       # Pale yellow
        RGB(0.8, 0.85, 0.7),      # Intermediate
        RGB(0.7, 0.9, 0.8),       # Cyan/turquoise highlights
        RGB(0.85, 0.95, 0.9),     # Intermediate
        RGB(1.0, 1.0, 1.0)        # Brilliant white for stars
    ]

    # Create a smoother gradient with fine transitions
    nebula = orion_nebula_gradient

    # Map the normalized values to color indices with smoother interpolation
    normalized = channelview(enhanced)
    normalized_clipped = clamp.(normalized, 0.0, 1.0)

    # Scale to gradient length with floating point precision (for smoother color transitions)
    color_indices_float = normalized_clipped .* (length(nebula) - 1) .+ 1

    # Create the color image using color interpolation
    color_image = Array{RGB{Float64}}(undef, size(normalized_clipped)...)

    for i in eachindex(normalized_clipped)
        idx_float = color_indices_float[i]
        idx_low = floor(Int, idx_float)
        idx_high = ceil(Int, idx_float)

        # Ensure indices are within bounds
        idx_low = clamp(idx_low, 1, length(nebula))
        idx_high = clamp(idx_high, 1, length(nebula))

        # Get interpolation factor
        t = idx_float - idx_low

        # Interpolate colors for smoother gradients
        if idx_low == idx_high
            color_image[i] = nebula[idx_low]
        else
            color_low = nebula[idx_low]
            color_high = nebula[idx_high]

            # Linear interpolation between colors
            r = (1-t) * color_low.r + t * color_high.r
            g = (1-t) * color_low.g + t * color_high.g
            b = (1-t) * color_low.b + t * color_high.b

            color_image[i] = RGB(r, g, b)
        end
    end

    # Apply slight sharpening for enhanced detail
    sharpened = imfilter(color_image, Kernel.gaussian(0.5))

    println("Saving image...")
    # Save both the original and a high-quality version
    save("buddhabrot_orion_detailed.png", transpose(color_image))
    save("buddhabrot_orion_detailed_sharp.png", transpose(sharpened))

    println("Done!")
end

buddhabrot()

SyntaxError: invalid decimal literal (<ipython-input-3-79e9067e455e>, line 12)

In [4]:
!julia

/bin/bash: line 1: julia: command not found


# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />